In [ ]:
!pip install transformers datasets torch

In [2]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

# Load the pretrained CodeT5-small model and tokenizer
model_name = "Salesforce/codet5-small"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [6]:
from datasets import load_dataset
dsVal = load_dataset("csv", data_files={"validation": "/content/validation.csv"})
dsTrain = load_dataset("csv", data_files={"train": "/content/train.csv"})
dsTest = load_dataset("csv", data_files={"test": "/content/test.csv"})

Generating validation split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
def preprocess_function(examples):
    # Use the most detailed context as input (src_fm_fc_ms_ff)
    inputs = ["generate unit test: " + ex for ex in examples["src_fm_fc_ms_ff"]]
    targets = examples["target"]

    # Tokenize inputs and targets
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    # Add tokenized labels to inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = dsTrain["train"].map(preprocess_function, batched=True)
validation_dataset = dsVal["validation"].map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./codet5-finetuned",
    evaluation_strategy="steps",
    save_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=100,
    eval_steps=500,
    save_steps=1000,
    warmup_steps=500,
    save_total_limit=2,
    weight_decay=0.01,
    fp16=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)

<ipython-input-9-84fc9e0aeb64>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
500,0.578600,0.660429
1000,0.535000,0.607330
1500,0.445100,0.593818
2000,0.429100,0.578950
2500,0.412300,0.575694
3000,0.413100,0.574268
3500,0.391600,0.571154


TrainOutput(global_step=3750, training_loss=0.5338582524617513, metrics={'train_runtime': 1817.0212, 'train_samples_per_second': 16.511, 'train_steps_per_second': 2.064, 'total_flos': 4060254044160000.0, 'train_loss': 0.5338582524617513, 'epoch': 3.0})

In [11]:
model.save_pretrained("./fine_tuned_codet5")
tokenizer.save_pretrained("./fine_tuned_codet5")


('./fine_tuned_codet5/tokenizer_config.json',
 './fine_tuned_codet5/special_tokens_map.json',
 './fine_tuned_codet5/vocab.json',
 './fine_tuned_codet5/merges.txt',
 './fine_tuned_codet5/added_tokens.json')

In [13]:
test_dataset = dsTest["test"].map(preprocess_function, batched=True)
test_dataset = test_dataset.remove_columns(["src_fm", "src_fm_fc", "src_fm_fc_co", "src_fm_fc_ms", "src_fm_fc_ms_ff", "target"])

In [14]:
results = trainer.evaluate(eval_dataset=test_dataset)
print("Test Results:", results)

Test Results: {'eval_loss': 0.5621889233589172, 'eval_runtime': 44.5396, 'eval_samples_per_second': 56.13, 'eval_steps_per_second': 7.027, 'epoch': 3.0}


In [15]:
!zip -r fine_tuned_codet5.zip fine_tuned_codet5/


  adding: fine_tuned_codet5/ (stored 0%)
  adding: fine_tuned_codet5/generation_config.json (deflated 34%)
  adding: fine_tuned_codet5/tokenizer_config.json (deflated 94%)
  adding: fine_tuned_codet5/merges.txt (deflated 54%)
  adding: fine_tuned_codet5/model.safetensors (deflated 7%)
  adding: fine_tuned_codet5/special_tokens_map.json (deflated 97%)
  adding: fine_tuned_codet5/config.json (deflated 61%)
  adding: fine_tuned_codet5/vocab.json (deflated 67%)
